<img src="http://openenergy-platform.org/static/OEP_logo_2_no_text.svg" alt="OpenEnergy Platform" height="100" width="100"  align="left"/>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/ad/Bhl_neue_wege_logo_transparent.svg/2000px-Bhl_neue_wege_logo_transparent.svg.png" alt="BHL" height="300" width="300" align="right"/>

# OpenEnergyPlatform
<br><br>

# MaStR Data Cleansing
Repository: https://github.com/OpenEnergyPlatform/data-preprocessing/tree/master/data-import/bnetza_mastr

Please report bugs and improvements here: https://github.com/OpenEnergyPlatform/data-preprocessing/issues <br>
How to get started with Jupyter Notebooks can be found here: https://github.com/OpenEnergyPlatform/oeplatform/wiki

In [ ]:
__copyright__ = "Bauhaus Luftfahrt e.V."
__license__   = "GNU Affero General Public License Version 3 (AGPL-3.0)"
__url__       = "https://github.com/openego/data_processing/blob/master/LICENSE"
__author__    = "Benjamin W. Portner"

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#OEP-MaStR-data" data-toc-modified-id="OEP-MaStR-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>OEP MaStR data</a></span><ul class="toc-item"><li><span><a href="#Load-data" data-toc-modified-id="Load-data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Load data</a></span></li><li><span><a href="#Drop-duplicates" data-toc-modified-id="Drop-duplicates-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Drop duplicates</a></span></li><li><span><a href="#Drop-outliers" data-toc-modified-id="Drop-outliers-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Drop outliers</a></span></li></ul></li><li><span><a href="#OPSD-data" data-toc-modified-id="OPSD-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>OPSD data</a></span><ul class="toc-item"><li><span><a href="#Load-and-clean-data" data-toc-modified-id="Load-and-clean-data-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Load and clean data</a></span></li><li><span><a href="#Match-datasets" data-toc-modified-id="Match-datasets-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Match datasets</a></span></li><li><span><a href="#Export" data-toc-modified-id="Export-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Export</a></span></li></ul></li></ul></div>

## OEP MaStR data

### Load data

First, let's load the datasets for hydro, wind and biomass:

In [ ]:
import pandas as pd

version = '1.4'

fn_wind = f'bnetza_mastr_rli_v{version}_wind'
df_wind = pd.read_csv(f'data/OEP/bnetza_mastr_power-units_rli_v{version}/{fn_wind}.csv', 
                      encoding='utf8', sep=';', parse_dates=["Inbetriebnahmedatum"], dtype={"Postleitzahl":str})

fn_hydro = f'bnetza_mastr_rli_v{version}_hydro'
df_hydro = pd.read_csv(f'data/OEP/bnetza_mastr_power-units_rli_v{version}/{fn_hydro}.csv', 
                       encoding='utf8', sep=';', parse_dates=["Inbetriebnahmedatum"], dtype={"Postleitzahl":str})

fn_biomass = f'bnetza_mastr_rli_v{version}_biomass'
df_biomass = pd.read_csv(f'data/OEP/bnetza_mastr_power-units_rli_v{version}/{fn_biomass}.csv', 
                         encoding='utf8', sep=';', parse_dates=["Inbetriebnahmedatum"], dtype={"Postleitzahl":str})

And merge them into one DataFrame:

In [ ]:
df_all = df_wind.append(df_hydro, ignore_index=True, sort=False).append(df_biomass, ignore_index=True, sort=False)
df_all.head()

For some reason, there is an unnamed column. Let's drop it:

In [ ]:
df_all.drop(columns=['Unnamed: 0'], inplace=True)
df_all.head()

### Drop duplicates

Next, let's see if all the entries are unique. Dropping duplicates does not work on this dataset:

In [ ]:
len_before = len(df_all)
len_after = len(df_all.drop_duplicates())

len_before, len_before == len_after

However, there are quite a few entries that are basically identical: 

In [ ]:
df_all[df_all.duplicated(subset="EinheitMastrNummer", keep=False)].sort_values(by="EinheitMastrNummer").head()

Why were these not dropped in pandas's "drop_duplicates"? Because the dataset contains a timestamp that documents the time of download for each entry. Naturally, that timestep is unique for each entry. Let's drop it and repeat:

In [ ]:
# create one dataframe with all duplicated entries
has_double = df_all.drop(columns=["timestamp_e"]).duplicated(keep=False)
df_duplicated = df_all[has_double].copy().sort_values(by="EinheitMastrNummer").reset_index()

# create one dataframe with only unique entries
is_unique = ~df_all.drop(columns=["timestamp_e"]).duplicated(keep="first")
df_unique = df_all[is_unique].copy().sort_values(by="EinheitMastrNummer").reset_index()

# compare sizes
len(df_all), len(df_duplicated), len(df_unique)

There are 65,133 unique entries. That is 6,004 less than in the merged dataset. There are 9,381 entries which have at least one double.

Let's check if everything worked alright:

In [ ]:
df_unique.head()

Looks good! Let's export the result:

In [ ]:
df_duplicated.to_csv("output/OEP_0_duplicated.csv", index=False)
df_unique.to_csv("output/OEP_0_unique.csv", index=False)

### Drop outliers
Some of the entries have coordinates which do not lie within Germany. Some of these are off-shore wind turbines. However, there are entries which have addresses in Bayern or Niedersachsen but are located in Italy, and even Africa. We want to remove these.

First off, imports:

In [ ]:
import geopandas as gpd
from shapely.geometry import Polygon
from shapely.geometry import Point

Now we can load some shape files. They contain the boundaries of Germany, Baltic sea, and North Sea:

In [ ]:
gdf_DE = gpd.read_file(r"data\shapefiles_germany\BKG\vg2500_sta.shp")
gdf_baltic = gpd.read_file(r"data\shapefiles_germany\marineregions.org\DE_baltic_sea\eez_iho.shp")
gdf_north_sea = gpd.read_file(r"data\shapefiles_germany\marineregions.org\DE_north_sea\eez_iho.shp")

gdf_DE

We need to extend the boundaries slightly to make sure that points close to Germany's border will be correctly identified:

In [ ]:
buffer = 0.01
gdf_DE["geometry"] = gdf_DE.buffer(buffer)
gdf_baltic["geometry"] = gdf_baltic.buffer(buffer)
gdf_north_sea["geometry"] = gdf_north_sea.buffer(buffer)

Now, to test if entries lie within German territory, we first need to convert them to a GeoDataFrame: 

In [ ]:
# drop entries without defined coordinates to prevent errors
df_no_na = df_unique[["EinheitMastrNummer","Laengengrad","Breitengrad"]].dropna()

# make list of shapely Points
points = [Point(xy) for xy in zip(df_no_na["Laengengrad"], df_no_na["Breitengrad"])]

# define coordinate system WGS 84
crs = {'init': 'epsg:4326'}

# make GeoDataFrame
gdf_points = gpd.GeoDataFrame(df_no_na["EinheitMastrNummer"], crs=crs, geometry=points)
gdf_points.head()

Finally, we can test for all points whether they are contained in each shape file:

In [ ]:
points_in_DE = gpd.sjoin(gdf_points, gdf_DE, how="right", op="within")
points_in_baltic = gpd.sjoin(gdf_points, gdf_baltic, how="right", op="within")
points_in_north_sea = gpd.sjoin(gdf_points, gdf_north_sea, how="right", op="within")

We define entries as located correctly if they are:
- wind, biomass or hydro on German soil
- wind in German parts of Baltic Sea or North Sea

In [ ]:
located_indices = (
    ( df_unique["EinheitMastrNummer"].isin(points_in_DE["EinheitMastrNummer"]) ) |
    (
            ( df_unique["Einheittyp"] == "Windeinheit" ) &
            (
                ( df_unique["EinheitMastrNummer"].isin(points_in_baltic["EinheitMastrNummer"]) ) |
                ( df_unique["EinheitMastrNummer"].isin(points_in_north_sea["EinheitMastrNummer"]) )
            )
    )
)
df_located = df_unique[located_indices].copy().reset_index()

All other entries are treated as unlocated:

In [ ]:
df_unlocated = df_unique[~located_indices].copy().reset_index()

How many entries are located / unlocated?

In [ ]:
len(df_unique), len(df_located), len(df_unlocated)

51% of all unique entries have not been located or have been wrongly located. 

Let's export located and unlocated entries as csv:

In [ ]:
df_located.to_csv("output/OEP_1_located.csv", index=False)
df_unlocated.to_csv("output/OEP_1_unlocated.csv", index=False)

Let's also create an interactive map of located and unlocated entries. We can zoom and pan the map. When hovering over a point on the map, the entry's name, address, type, and gross output is shown as a tooltip. Also, clicking on the legend allows hiding and showing different types of powerplants.

The plotting logic is contained in a separate function "plotPowerPlants" in the module "helpers":

In [ ]:
from helpers import *

plot_located = plotPowerPlants(df_located)

# export as html
gv.renderer("bokeh").save(plot_located, "output/OEP_1_located")

# figure is very big (5 MB)! plotting in jupyter is not recommended!
#plot_located

Let's also plot the unlocated entries to check if everything worked alright:

In [ ]:
plot_unlocated = plotPowerPlants(df_unlocated)

# export as html
gv.renderer("bokeh").save(plot_unlocated, "output/OEP_1_unlocated")

# show in notebook
plot_unlocated

Looks like the outliers were identified correctly. Now, what do we do about these and about unlocated entries? Maybe we can use data from other sources to get their coordinates.

## OPSD data

Open power Systems Data (OPSD, https://open-power-system-data.org/) is another project offering power plant data for Germany (and many other European states). The datasets include powerplant type, generation capacity, and geo-coordinates (among other fields). 

### Load and clean data

There renewable plants dataset includes many different power plant types, including hydro, solar, wind, bioenergy, and geothermal. For this analysis, I will keep only those contained in the OEP dataset: hydro, bioenergy, and wind. Also, I will rename some columns for compatibility with the OEP MaStR data.

In [ ]:
# load renewables
df_renewables = pd.read_csv(
    "data/OPSD/renewable_power_plants_DE.csv",
    sep=";", encoding="ANSI", parse_dates=["commissioning_date"]
)

# keep only wind, bioenergy and hydro
df_renewables = df_renewables[df_renewables["energy_source_level_2"].isin(["Wind","Bioenergy","Hydro"])]

# rename columns for compatibiliy with OEP data
df_renewables.rename(columns={
    "energy_source_level_2": "Einheittyp",
    "address" : "Standort",
    "federal_state": "Bundesland",
    "commissioning_date": "Inbetriebnahmedatum",
    "electrical_capacity": "Bruttoleistung",
    "lat": "Breitengrad",
    "lon": "Laengengrad",
    "eeg_id": "AnlagenschluesselEeg",
}, inplace=True)

# add columns which are not defined in OPSD data
df_renewables["Name"] = None
df_renewables["Land"] = "Deutschland"

# rename types for compatibility with OEP
df_renewables["Einheittyp"].replace({"Hydro": "Wasser", "Bioenergy": "Biomasse", "Wind": "Windeinheit"}, inplace=True)

df_renewables.head()

Strangely, some biomass and hydro plants are contained only in the fossil power plant dataset. Let's load these as well.

In [ ]:
# load conventional
df_conv = pd.read_csv(
    "data/OPSD/conventional_power_plants_DE.csv",
    sep = ",", encoding = "ANSI", decimal=".", dtype={"commissioned": str}
)

# keep only wind, bioenergy and hydro
df_conv = df_conv[df_conv["energy_source_level_2"].isin(["Wind","Bioenergy","Hydro"])]

# commissioning dates are given as string with format YYYY.0 - Parse manually:
df_conv["commissioned"] = pd.to_datetime(df_conv["commissioned"], format="%Y.0")

# rename columns for compatibiliy with OEP data
df_conv.rename(columns={
    "energy_source_level_2": "Einheittyp",
    "street": "Standort",
    "state": "Bundesland",
    "commissioned": "Inbetriebnahmedatum",
    "capacity_gross_uba": "Bruttoleistung",
    "lat": "Breitengrad",
    "lon": "Laengengrad",
    "eeg": "AnlagenschluesselEeg",
}, inplace=True)

# add columns which are not defined in OPSD data
df_conv["Land"] = "Deutschland"
df_conv["Name"] = df_conv["name_bnetza"].fillna("") + df_conv["name_uba"].fillna("")

# rename types for compatibility with OEP
df_conv["Einheittyp"].replace({"Hydro": "Wasser", "Bioenergy": "Biomasse", "Wind": "Windeinheit"}, inplace=True)

# merge
df_OPSD = df_renewables.append(df_conv, ignore_index=True, sort=False)

df_OPSD.head()

How many entries do we have? Let's compare to the OEP data.

In [ ]:
opsd = df_OPSD["Einheittyp"].value_counts()
oep = df_unique["Einheittyp"].value_counts()
df = pd.DataFrame(columns=opsd.index, data=[opsd.values, oep.values], index=["OPSD", "OEP"])
df["total"] = df.sum(axis=1)
df

The OEP MaStR dataset is more than three times larger! Still, we can extract some new coordinates from the OPSD data.

### Match datasets

OPSD datasets contain columns "eeg_id" / "eeg" which are identical to OEP-MaStR dataset column "AnlagenschluesselEeg". I will use these columns for matching. To simplify matching, the OPSD columns have already been renamed to their OEP counter-part during loading.

First, I need to remove entries without an EEG ID from both datasets.

In [ ]:
df_OPSD_EEG = df_OPSD[~df_OPSD["AnlagenschluesselEeg"].isna()]
df_OEP_EEG = df_unique[~df_unique["AnlagenschluesselEeg"].isna()]

Let's compare the dataset sizes again.

In [ ]:
opsd = df_OPSD_EEG["Einheittyp"].value_counts()
oep = df_OEP_EEG["Einheittyp"].value_counts()
df = pd.DataFrame(columns=opsd.index, data=[opsd.values, oep.values], index=["OPSD", "OEP"])
df["total"] = df.sum(axis=1)
df

Luckily, most entries in both datasets have an EEG ID. Next, let's find the intersection.

In [ ]:
intersecting_EEGs = set(df_OPSD_EEG["AnlagenschluesselEeg"]).intersection(set(df_OEP_EEG["AnlagenschluesselEeg"]))
len(intersecting_EEGs)

13727 entries are contained in both the OEP and in the OPSD data. This means, the OEP MaStR dataset contains 73% of the OPSD biomass, wind and hydro data.

Now, how many entries in the intersection are unlocated in the OEP dataset?

In [ ]:
df_OEP_intersected_unlocated = df_unlocated[df_unlocated["AnlagenschluesselEeg"].isin(intersecting_EEGs)]
len(df_OEP_intersected_unlocated)

1134 entries! Let's try to fetch their coordinates from the OPSd dataset. 

Note: The EEG ID is not unique, so I cannot use normal merge/join operations!

In [ ]:
df_OPSD_intersected_located = df_OPSD_EEG[
    ( df_OPSD_EEG["AnlagenschluesselEeg"].isin(df_OEP_intersected_unlocated["AnlagenschluesselEeg"]) ) &
    ~(
        df_OPSD_EEG["Laengengrad"].isna() | 
        df_OPSD_EEG["Breitengrad"].isna() 
    )
]

df_extracted = df_OEP_intersected_unlocated.copy()

for OPSD_index, eeg_id in df_OPSD_intersected_located["AnlagenschluesselEeg"].items():
    OEP_indices = df_extracted[df_extracted["AnlagenschluesselEeg"]==eeg_id].index
    df_extracted.loc[OEP_indices, ["Laengengrad", "Breitengrad"]] = \
        df_OPSD_intersected_located.loc[OPSD_index,["Laengengrad", "Breitengrad"]].values

df_extracted.dropna(subset=["Laengengrad", "Breitengrad"], inplace=True)

len(df_extracted)

We could extract 1133 coordinates. All but one OPSD entry had coordinates associated with them. 

### Export

Let's export the dataframes as csv.

Newly located entries.

In [ ]:
df_extracted.to_csv("output/OEP_2_located_only_OPSD.csv", index=False)

Let's also update the list of located / unlocated entries.

In [ ]:
df_OEP_OPSD_located = df_located.append(df_extracted, ignore_index=True)
df_OEP_OPSD_located.to_csv("output/OEP_2_located_incl_OPSD.csv", index=False)

df_OEP_OPSD_unlocated = df_unique[
    ~( df_unique["EinheitMastrNummer"].isin(df_OEP_OPSD_located["EinheitMastrNummer"]) )
].copy()
df_OEP_OPSD_unlocated.to_csv("output/OEP_2_unlocated_incl_OPSD.csv", index=False)

len(df_OEP_OPSD_located), len(df_OEP_OPSD_unlocated)

More than half of the entries are now located! Let's plot.

Newly located entries (OPSD intersection).

In [ ]:
plot_OPSD = plotPowerPlants(df_extracted)
gv.renderer("bokeh").save(plot_OPSD, "output/OEP_2_located_only_OPSD")
plot_OPSD

All located entries (OPSD intersection + OEP).

In [ ]:
plot_located = plotPowerPlants(df_OEP_OPSD_located)
gv.renderer("bokeh").save(plot_located, "output/OEP_2_located_incl_OPSD")

# figure is very big (5 MB)! plotting in jupyter is not recommended!
#plot_located

Outliers.

In [ ]:
plot_unlocated = plotPowerPlants(df_OEP_OPSD_unlocated)
gv.renderer("bokeh").save(plot_unlocated, "output/OEP_2_unlocated_incl_OPSD")
plot_unlocated

There are still some outliers which could not be corrected using OPSD data.